# ML Classifers
- KNN
- Random Forest
- SVM
- AdaBoost

## Classifer: Random Forest

In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
import pickle

#read final dataset .csv file
full_feature_df = pd.read_csv("preprocess_data.csv")
full_feature_df.head(2)

GENDER_1  GENDER_2  RACE_1  RACE_2  RACE_3  RACE_4  RACE_5  RACE_6  RACE_7  \
0         0         1       1       0       0       0       0       0       0   
1         0         1       1       0       0       0       0       0       0   

   RACE_8  ...  B2SENSTI_1.0  B2SENSTI_2.0  B2SENSTI_3.0  B2SENSTI_4.0  \
0       0  ...             0             0             1             0   
1       0  ...             0             0             1             0   

   B2SENSTI_5.0  P7ATTENI  P7BEHAVE  P7SOLVE  P6SAMEAG  P4SAMEAG  
0             0       1.0       1.0      1.0       1.0       0.0  
1             0       1.0       1.0      0.0       1.0       1.0  

[2 rows x 108 columns]

In [2]:
reduce_feature_df = pd.read_csv("reduced_FS.csv")
reduce_feature_df.head(2)

P2HITFUN_1.0  P2HITFUN_2.0  P2WARMCL_4.0  P2EXPRES_3.0  P2EXPRES_2.0  \
0             0             1             0             0             0   
1             0             1             0             0             0   

   P2CHLIKE_2.0  P2CHLIKE_1.0  P2EXPRES_1.0  P2WARMCL_2.0  P2WARMCL_1.0  \
0             0             1             1             0             1   
1             0             1             1             0             1   

   P2HITIGN_1.0  P2HITIGN_2.0  P4OPINIO_1.0  P2CHLIKE_3.0  P2WARMCL_3.0  \
0             0             1             0             0             0   
1             0             1             0             0             0   

   P2HITCHO_1.0  P2HITCHO_2.0  P2CHLIKE_4.0  P4LISTEN_2.0  P2FEELAN_1.0  
0             0             1             0             0             0  
1             0             1             0             0             0

In [ ]:
# spliting the target labels from descriptive features
X = full_feature_df.drop(columns=full_feature_df.columns[-5:]) #removes the target labels
targets = full_feature_df.iloc[:, -5:] #get the target labels 
#fix the ranking to 1-4
for label in targets.columns:
    unique_values = targets[label].unique()

    # Create a mapping to increment each unique value by 1
    value_mapping = {val: val + 1 for val in unique_values}
    #print("Value mapping:", value_mapping)

    # Apply the mapping to the 'target' column
    targets[label] = targets[label].map(value_mapping)

for column in targets.columns:
    print(f"Class distribution for {column}:")
    print(targets[column].value_counts())
    print("-" * 30)

# KNN (full features)

In [5]:


# File names for KNN full feature set
file_names = ['finalized_model_M1.sav', 'finalized_model_M2.sav', 
              'finalized_model_M3.sav', 'finalized_model_M4.sav', 
              'finalized_model_M5.sav']

# Ensure target labels are integers
targets = targets.astype(int)

# Index for file_names
index = 0
# Iterate over each target label
for index, target_label in enumerate(targets.columns):
    if index >= len(file_names):
        print("Not enough filenames provided for all target labels.")
        break

    y = targets[target_label]  # Select specific target label

    # Check class diversity in the target label
    class_counts = y.value_counts()
    if len(class_counts) <= 1:
        print(f"Skipping {target_label} due to lack of class diversity: {class_counts}")
        continue

    # Balance the dataset using SMOTE
    try:
        smote = SMOTE(random_state=42)
        X_balanced, y_balanced = smote.fit_resample(X, y)
    except ValueError as e:
        print(f"Error during SMOTE for {target_label}: {e}")
        continue

    # Split the balanced data
    X_train, X_test, y_train, y_test = train_test_split(
        X_balanced, y_balanced, test_size=0.2, random_state=42
    )

    # Create and train the KNN model
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train, y_train)
    
    # Save the model to disk
    filename = file_names[index]
    with open(filename, 'wb') as file:
        pickle.dump(knn, file)

    # Load the model from disk (optional)
    loaded_model = pickle.load(open(filename, 'rb'))
    result = loaded_model.score(X_test, y_test)

    # Print the score for this target label
    print(f"Score for {filename} ({target_label}): {result}")
    

Score for finalized_model_M1.sav (P7ATTENI): 0.8428167459824816
Score for finalized_model_M2.sav (P7BEHAVE): 0.8614681330329932
Score for finalized_model_M3.sav (P7SOLVE): 0.8280063064354307
Score for finalized_model_M4.sav (P6SAMEAG): 0.8391160380074864
Score for finalized_model_M5.sav (P4SAMEAG): 0.8233003660151078


# KNN (reduced set)

In [ ]:
# File names for KNN reduced feature set
file_names = ['finalized_model_M6.sav', 'finalized_model_M7.sav', 
              'finalized_model_M8.sav', 'finalized_model_M9.sav', 
              'finalized_model_M10.sav']

#get reduced features 
X = reduce_feature_df

# Ensure target labels are integers
targets = targets.astype(int)

# Index for file_names
index = 0

#get the target labels
targets = full_feature_df.iloc[:, -5:]  

# Iterate over each target label
for index, target_label in enumerate(targets.columns):
    if index >= len(file_names):
        print("Not enough filenames provided for all target labels.")
        break

    y = targets[target_label]  # Select specific target label

    # Check class diversity in the target label
    class_counts = y.value_counts()
    if len(class_counts) <= 1:
        print(f"Skipping {target_label} due to lack of class diversity: {class_counts}")
        continue
        
    # Balance the dataset using SMOTE
    try:
        smote = SMOTE(random_state=42)
        X_balanced, y_balanced = smote.fit_resample(X, y)
    except ValueError as e:
        print(f"Error during SMOTE for {target_label}: {e}")
        continue

    # Split the balanced data
    X_train, X_test, y_train, y_test = train_test_split(
        X_balanced, y_balanced, test_size=0.2, random_state=42
    )
    print(f"the TEST size of 20% is {len(y_test)}\n")

    # Create and train the KNN model
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train, y_train)
    
    # Save the model to disk
    filename = file_names[index]
    with open(filename, 'wb') as file:
        pickle.dump(knn, file)

    # Load the model from disk (optional)
    loaded_model = pickle.load(open(filename, 'rb'))
    result = loaded_model.score(X_test, y_test)

    # Print the score for this target label
    print(f"Score for {filename} ({target_label}): {result}")


# Random Forest (full features)

In [3]:
from sklearn.ensemble import RandomForestClassifier

# File names for the Random Forest full features
file_names = ['finalized_model_M11.sav', 'finalized_model_M12.sav', 'finalized_model_M13.sav', 'finalized_model_M14.sav', 
              'finalized_model_M15.sav']

#get full features 
X = full_feature_df.drop(columns=full_feature_df.columns[-5:]) #removes the target labels

#get the target labels
targets = full_feature_df.iloc[:, -5:]  

for label in targets.columns:
    unique_values = sorted(targets[label].unique())  # Get sorted unique values
    value_mapping = {val: rank for rank, val in enumerate(unique_values, start=1)}  # Create rank mapping (1–4)
    targets[label] = targets[label].map(value_mapping)  # Apply the rank mapping

# Index for file_names
index = 0  

# Ensure target labels are integers
targets = targets.astype(int)

# Iterate over each target label
for index, target_label in enumerate(targets.columns):
    if index >= len(file_names):
        print("Not enough filenames provided for all target labels.")
        break
        
    y = targets[target_label]  # Extract target values for the current column

    # Check class diversity in the target label
    class_counts = y.value_counts()
    if len(class_counts) <= 1:
        print(f"Skipping {target_label} due to lack of class diversity: {class_counts}")
        continue

    # Balance the dataset using SMOTE
    try:
        smote = SMOTE(random_state=42)
        X_balanced, y_balanced = smote.fit_resample(X, y)
    except ValueError as e:
        print(f"Error during SMOTE for {target_label}: {e}")
        continue

    # Split the balanced data
    X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)
    

    # Create Random Forest classifier instance
    rf = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust n_estimators and other parameters
    
    # Train the model on the training data
    rf.fit(X_train, y_train)
    
    #Save the model to disk using pickle
    filename = file_names[index]
    pickle.dump(rf, open(filename, 'wb'))
    
    # Save the model to disk
    filename = file_names[index]
    with open(filename, 'wb') as file:
        pickle.dump(rf, file)

    # Load the model from disk (optional)
    loaded_model = pickle.load(open(filename, 'rb'))
    result = loaded_model.score(X_test, y_test)

    # Print the score for this target label
    print(f"Score for {filename} ({target_label}): {result}\n")
    

Score for finalized_model_M11.sav (P7ATTENI): 0.9013725084488585

Score for finalized_model_M12.sav (P7BEHAVE): 0.9250695640651915

Score for finalized_model_M13.sav (P7SOLVE): 0.893077253834026

Score for finalized_model_M14.sav (P6SAMEAG): 0.8962712352433055

Score for finalized_model_M15.sav (P4SAMEAG): 0.863406276769722



# Random Forest (reduced features)

In [9]:
# File names for the Random Forest reduced features
file_names = ['finalized_model_M16.sav', 'finalized_model_M17.sav', 'finalized_model_M18.sav', 'finalized_model_M19.sav', 
              'finalized_model_M20.sav']

#get reduced features 
X = reduce_feature_df

#get the target labels
targets = full_feature_df.iloc[:, -5:]  

for label in targets.columns:
    unique_values = sorted(targets[label].unique())  # Get sorted unique values
    value_mapping = {val: rank for rank, val in enumerate(unique_values, start=1)}  # Create rank mapping (1–4)
    targets[label] = targets[label].map(value_mapping)  # Apply the rank mapping
# Index for file_names
index = 0  

# Ensure target labels are integers
targets = targets.astype(int)

# Iterate over each target label
for index, target_label in enumerate(targets.columns):
    if index >= len(file_names):
        print("Not enough filenames provided for all target labels.")
        break
    # Extract target values for the current column
    y = targets[target_label]  

    # Check class diversity in the target label
    class_counts = y.value_counts()
    if len(class_counts) <= 1:
        print(f"Skipping {target_label} due to lack of class diversity: {class_counts}")
        continue

    # Balance the dataset using SMOTE
    try:
        smote = SMOTE(random_state=42)
        X_balanced, y_balanced = smote.fit_resample(X, y)
    except ValueError as e:
        print(f"Error during SMOTE for {target_label}: {e}")
        continue

    # Split the balanced data
    X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)
    
    print(f"the TEST size of 20% is {len(y_test)}\n")

    # Create Random Forest classifier instance
    rf = RandomForestClassifier(n_estimators=100, random_state=42)  
    
    # Train the model on the training data
    rf.fit(X_train, y_train)

    #Save the model to disk using pickle
    filename = file_names[index]
    pickle.dump(rf, open(filename, 'wb'))
    
    # Save the model to disk
    filename = file_names[index]
    with open(filename, 'wb') as file:
        pickle.dump(rf, file)

    # Load the model from disk (optional)
    loaded_model = pickle.load(open(filename, 'rb'))
    result = loaded_model.score(X_test, y_test)

    # Print the score for this target label
    print(f"Score for {filename} ({target_label}): {result}\n")
    

the TEST size of 20% is 14499

Score for finalized_model_M16.sav (P7ATTENI): 0.3026415614869991

the TEST size of 20% is 15094

Score for finalized_model_M17.sav (P7BEHAVE): 0.32940241155426

the TEST size of 20% is 13954

Score for finalized_model_M18.sav (P7SOLVE): 0.31711337250967464

the TEST size of 20% is 13892

Score for finalized_model_M19.sav (P6SAMEAG): 0.3152893751799597

the TEST size of 20% is 12841

Score for finalized_model_M20.sav (P4SAMEAG): 0.3182773927264232



# Simple Vector Machine (Full Features)

In [4]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pickle

# File names for the SVM full features
file_names = ['finalized_model_M21.sav', 'finalized_model_M22.sav', 'finalized_model_M23.sav', 'finalized_model_M24.sav', 
              'finalized_model_M25.sav']

#removes the target labels
X = full_feature_df.drop(columns=full_feature_df.columns[-5:]) 

# Ensure target labels are integers
targets = full_feature_df.iloc[:, -5:].astype(int)  

for label in targets.columns:
    unique_values = sorted(targets[label].unique())  # Get sorted unique values
    value_mapping = {val: rank for rank, val in enumerate(unique_values, start=1)}  # Create rank mapping (1–4)
    targets[label] = targets[label].map(value_mapping)  # Apply the rank mapping
#filename index
index = 0

# Iterate over each target label
for index, target_label in enumerate(targets.columns):
    if index >= len(file_names):
        print("Not enough filenames provided for all target labels.")
        break
    # Extract target values for the current column    
    y = targets[target_label]  
    
    # Check class diversity in the target label
    class_counts = y.value_counts()
    if len(class_counts) <= 1:
        print(f"Skipping {target_label} due to lack of class diversity: {class_counts}")
        continue

    # Balance the dataset using SMOTE
    try:
        smote = SMOTE(random_state=42)
        X_balanced, y_balanced = smote.fit_resample(X, y)
    except ValueError as e:
        print(f"Error during SMOTE for {target_label}: {e}")
        continue

    # Split the balanced data
    X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)
    
    # Create SVM classifier instance
    svm = SVC(kernel='rbf', random_state=42) 
    
    # Train the model on the training data
    svm.fit(X_train, y_train)

    # Save the model to disk using pickle
    filename = file_names[index]
    with open(filename, 'wb') as file:
        pickle.dump(svm, file)

    # Load the model from disk (optional)
    loaded_model = pickle.load(open(filename, 'rb'))
    result = loaded_model.score(X_test, y_test)

    # Print the score for this target label
    print(f"Score for {filename} ({target_label}): {result}\n")


Score for finalized_model_M21.sav (P7ATTENI): 0.7859162700875922

Score for finalized_model_M22.sav (P7BEHAVE): 0.8525241817940904

Score for finalized_model_M23.sav (P7SOLVE): 0.8060771105059481

Score for finalized_model_M24.sav (P6SAMEAG): 0.8296861503023323

Score for finalized_model_M25.sav (P4SAMEAG): 0.8026633439763259



# SVM (Reduced Features)

In [4]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pickle

# File names for the SVM reduced features
file_names = ['finalized_model_M26.sav', 'finalized_model_M27.sav', 'finalized_model_M28.sav', 'finalized_model_M29.sav', 
              'finalized_model_M30.sav']
#removes the target labels
X = reduce_feature_df 

# Ensure target labels are integers
targets = full_feature_df.iloc[:, -5:].astype(int)  # Adjusted if needed

for label in targets.columns:
    unique_values = sorted(targets[label].unique())  # Get sorted unique values
    value_mapping = {val: rank for rank, val in enumerate(unique_values, start=1)}  # Create rank mapping (1–4)
    targets[label] = targets[label].map(value_mapping)  # Apply the rank mapping
#filename index
index = 0

# Iterate over each target label
for index, target_label in enumerate(targets.columns):
    if index >= len(file_names):
        print("Not enough filenames provided for all target labels.")
        break
    # Extract target values for the current column    
    y = targets[target_label]  
    
    # Check class diversity in the target label
    class_counts = y.value_counts()
    if len(class_counts) <= 1:
        print(f"Skipping {target_label} due to lack of class diversity: {class_counts}")
        continue

    # Balance the dataset using SMOTE
    try:
        smote = SMOTE(random_state=42)
        X_balanced, y_balanced = smote.fit_resample(X, y)
    except ValueError as e:
        print(f"Error during SMOTE for {target_label}: {e}")
        continue

    # Split the balanced data
    X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)
    
    # Create SVM classifier instance
    svm = SVC(kernel='rbf', random_state=42)
    
    # Train the model on the training data
    svm.fit(X_train, y_train)

    # Save the model to disk using pickle
    filename = file_names[index]
    with open(filename, 'wb') as file:
        pickle.dump(svm, file)

    # Load the model from disk (optional)
    loaded_model = pickle.load(open(filename, 'rb'))
    result = loaded_model.score(X_test, y_test)

    # Print the score for this target label
    print(f"Score for {filename} ({target_label}): {result}\n")

The TEST size of 20% is 14499

Score for finalized_model_M26.sav (P7ATTENI): 0.29326160424856884

The TEST size of 20% is 15094

Score for finalized_model_M27.sav (P7BEHAVE): 0.3136345567775275

The TEST size of 20% is 13954

Score for finalized_model_M28.sav (P7SOLVE): 0.29704744159380825

The TEST size of 20% is 13892

Score for finalized_model_M29.sav (P6SAMEAG): 0.3013964871868701

The TEST size of 20% is 12841

Score for finalized_model_M30.sav (P4SAMEAG): 0.2956156062611946



# ADABoost (Full Features)

In [9]:
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import pandas as pd

# File names for the AdaBoost full features
file_names = ['finalized_model_M31.sav', 'finalized_model_M32.sav', 'finalized_model_M33.sav', 'finalized_model_M34.sav', 
              'finalized_model_M35.sav']


# Prepare the dataset (assuming full_feature_df is a pandas DataFrame)
 # Removes the target labels
X = full_feature_df.drop(columns=full_feature_df.columns[-5:]) 
 # Extract the target labels
targets = full_feature_df.iloc[:, -5:].astype(int) 

for label in targets.columns:
    unique_values = sorted(targets[label].unique())  # Get sorted unique values
    value_mapping = {val: rank for rank, val in enumerate(unique_values, start=1)}  # Create rank mapping (1–4)
    targets[label] = targets[label].map(value_mapping)  # Apply the rank mapping
    
#le = LabelEncoder()

#filename index
index = 0
# Iterate through each target label
for index, target_label in enumerate(targets.columns):
    # Ensure there are enough filenames if applicable
    if index >= len(file_names):
        print("Not enough filenames provided for all target labels.")
        break

     # Extract target values for the current column
    y = targets[target_label] 
    #y = le.fit_transform(y)

    # Check class diversity in the target label
    class_counts = pd.Series(y).value_counts()
    if len(class_counts) <= 1:
        print(f"Skipping {target_label} due to lack of class diversity: {class_counts}")
        continue

    # Balance the dataset using SMOTE
    try:
        smote = SMOTE(random_state=42)
        X_balanced, y_balanced = smote.fit_resample(X, y)
    except ValueError as e:
        print(f"Error during SMOTE for {target_label}: {e}")
        continue

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X_balanced, y_balanced, stratify=y_balanced, random_state=0
    )

    # Train a Decision Tree as the base classifier for AdaBoost
    base_tree = DecisionTreeClassifier(max_depth=20, random_state=42)
    adaboost = AdaBoostClassifier(
        estimator=base_tree,
        n_estimators=50,
        learning_rate=1.0,
        algorithm='SAMME',  
        random_state=42
    )
    adaboost.fit(X_train, y_train)

    filename = file_names[index]
    with open(filename, 'wb') as file:
        pickle.dump(adaboost, file)

    loaded_model = pickle.load(open(filename, 'rb'))
    result = loaded_model.score(X_test, y_test)

    # Print the score for this target label
    print(f"Score for {filename} ({target_label}): {result}\n")

Score for finalized_model_M31.sav (P7ATTENI): 0.8575842851625007

Accuracy: 0.86

Classification Report:
               precision    recall  f1-score   support

           1       0.80      0.79      0.79      4531
           2       0.78      0.79      0.79      4531
           3       0.87      0.87      0.87      4530
           4       0.98      0.98      0.98      4531

    accuracy                           0.86     18123
   macro avg       0.86      0.86      0.86     18123
weighted avg       0.86      0.86      0.86     18123

------------------------------
Score for finalized_model_M32.sav (P7BEHAVE): 0.9016801823289341

Accuracy: 0.90

Classification Report:
               precision    recall  f1-score   support

           1       0.85      0.81      0.83      4717
           2       0.81      0.85      0.83      4717
           3       0.95      0.95      0.95      4716
           4       0.99      0.99      0.99      4717

    accuracy                           0.90     18

# AdaBoost (Reduced Features) 

In [17]:
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import pandas as pd
import pickle

# File names for the AdaBoost reduced features 
file_names = ['finalized_model_M36.sav', 'finalized_model_M37.sav', 'finalized_model_M38.sav', 'finalized_model_M39.sav', 
              'finalized_model_M40.sav']

# Prepare the dataset (assuming reduce_feature_df and full_feature_df are pandas DataFrames)
X = reduce_feature_df  # Reduced feature set
targets = full_feature_df.iloc[:, -5:].astype(int)  # Extract the target labels

le = LabelEncoder()

#filename index
index = 0

# Iterate through each target label
for index, target_label in enumerate(targets.columns):
    # Ensure there are enough filenames if applicable
    if index >= len(file_names):
        print("Not enough filenames provided for all target labels.")
        break

    y = targets[target_label]  # Extract target values for the current column
    y = le.fit_transform(y)

    # Check class diversity in the target label
    class_counts = pd.Series(y).value_counts()
    if len(class_counts) <= 1:
        print(f"Skipping {target_label} due to lack of class diversity: {class_counts}")
        continue

    # Balance the dataset using SMOTE
    try:
        smote = SMOTE(random_state=42)
        X_balanced, y_balanced = smote.fit_resample(X, y)
    except ValueError as e:
        print(f"Error during SMOTE for {target_label}: {e}")
        continue

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X_balanced, y_balanced, stratify=y_balanced, random_state=0
    )

    # Train a Decision Tree as the base classifier for AdaBoost
    base_tree = DecisionTreeClassifier(max_depth=1, random_state=42)
    adaboost = AdaBoostClassifier(
        estimator=base_tree,
        n_estimators=50,
        learning_rate=1.0,
        algorithm='SAMME',
        random_state=42
    )
    adaboost.fit(X_train, y_train)

    # Save the AdaBoost model to a file
    filename = file_names[index]
    with open(filename, 'wb') as file:
        pickle.dump(adaboost, file)

    loaded_model = pickle.load(open(filename, 'rb'))
    result = loaded_model.score(X_test, y_test)

    # Print the score for this target label
    print(f"Score for {filename} ({target_label}): {result}\n")


Score for finalized_model_M36.sav (P7ATTENI): 0.275671798267395

Score for finalized_model_M37.sav (P7BEHAVE): 0.26501298563629616

Score for finalized_model_M38.sav (P7SOLVE): 0.2624125673661277

Score for finalized_model_M39.sav (P6SAMEAG): 0.26651310106536136

Score for finalized_model_M40.sav (P4SAMEAG): 0.2589869790044234

